In [1]:
#Analysis
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import xlwings as xw
#Visulization
from matplotlib import pyplot
import seaborn as sns
#Main Model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import xgboost
import lightgbm as lgb
import scipy
from sklearn.metrics import roc_auc_score, roc_curve

Import Data

In [2]:
train_df = pd.read_csv('D:\\Kaggler\\Santander\\train.csv')
test_df = pd.read_csv('D:\\Kaggler\\Santander\\test.csv')
print("Train set-(Rows, Columns) : ",train_df.shape)
print("Test set-(Rows, Columns) : ",test_df.shape)

Train set-(Rows, Columns) :  (200000, 202)
Test set-(Rows, Columns) :  (200000, 201)


In [3]:
#Simple Data type Check
print("Train_df:")
print(train_df.head())
#print(train_df.describe())
#print(sum(train_df['target'])/len(train_df['target']))
print(train_df.corr())
#print("Test_df:")
#print(test_df.head())
#print(test_df.describe())

Train_df:
   ID_code  target    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427   
3  train_3       0  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428   
4  train_4       0   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405   

     var_7   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227  -2.3978   
1  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   2.0339   
2  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   3.1417   
3  14.9250   ...      4.4666   4.7433   0.7178   1.4214  23.0347  -1.2706   
4  19.2514   ...     -1.4905   9.5214  -0.1508   9.1942  13.2876  -1.5121   

   var_196  var_197  var_198  var_199  


In [7]:
train_df.corr().abs().unstack().sort_values()

var_191  var_75     2.703975e-08
var_75   var_191    2.703975e-08
var_173  var_6      5.942735e-08
var_6    var_173    5.942735e-08
var_126  var_109    1.313947e-07
var_109  var_126    1.313947e-07
var_144  var_27     1.772502e-07
var_27   var_144    1.772502e-07
var_177  var_100    3.116544e-07
var_100  var_177    3.116544e-07
var_12   var_181    4.033775e-07
var_181  var_12     4.033775e-07
var_116  var_150    6.628008e-07
var_150  var_116    6.628008e-07
var_142  var_44     7.934263e-07
var_44   var_142    7.934263e-07
var_13   var_171    1.181142e-06
var_171  var_13     1.181142e-06
var_69   var_126    1.218374e-06
var_126  var_69     1.218374e-06
var_173  var_176    1.318335e-06
var_176  var_173    1.318335e-06
var_55   var_119    1.356175e-06
var_119  var_55     1.356175e-06
var_175  var_65     1.398626e-06
var_65   var_175    1.398626e-06
var_119  var_76     1.605729e-06
var_76   var_119    1.605729e-06
var_84   var_2      1.699221e-06
var_2    var_84     1.699221e-06
          

Pre model step

In [8]:
column_list=list(train_df.columns)
column_list.remove('ID_code')
column_list.remove('target')
features = column_list

In [ ]:
#for feature in features:
    #train_df['Order'+feature]=scipy.stats.rankdata(train_df[feature])
    #test_df['Order'+feature]=scipy.stats.rankdata(test_df[feature])

In [9]:
#column_list=list(train_df.columns)
#column_list.remove('ID_code')
#column_list.remove('target')
#features = column_list
X = train_df[features]
X.head()    

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [10]:
y=train_df['target']

Split date into Train and Test

In [ ]:
model = XGBClassifier()
n_estimators = [50, 100, 150, 200]
max_depth = [2, 4, 6, 8]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X, label_encoded_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot results
scores = numpy.array(means).reshape(len(max_depth), len(n_estimators))
for i, value in enumerate(max_depth):
    pyplot.plot(n_estimators, scores[i], label='depth: ' + str(value))
pyplot.legend()
pyplot.xlabel('n_estimators')
pyplot.ylabel('Log Loss')
pyplot.savefig('n_estimators_vs_max_depth.png')

In [11]:
#Split date into Train and Test
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1064)

In [14]:
d_train = xgboost.DMatrix(train_X, label=train_y)
d_valid = xgboost.DMatrix(val_X, label=val_y)

In [15]:
params = {
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'learning_rate': 0.0936165921314771,
 'max_depth': 2,
 'colsample_bytree': 0.3561271102144279,
 'subsample': 0.8246604621518232,
 'min_child_weight': 53,
 'gamma': 9.943467991283027,
 'silent': 1}

In [17]:
Model = xgboost.train(params=params, dtrain=d_train, num_boost_round=4000, evals=[(d_train, "Train"), (d_valid, "Val")],verbose_eval= 100, early_stopping_rounds=50) 

[0]	Train-auc:0.550647	Val-auc:0.542765
Multiple eval metrics have been passed: 'Val-auc' will be used for early stopping.

Will train until Val-auc hasn't improved in 50 rounds.
[100]	Train-auc:0.828302	Val-auc:0.805869
[200]	Train-auc:0.865588	Val-auc:0.841584
[300]	Train-auc:0.883832	Val-auc:0.857971
[400]	Train-auc:0.894608	Val-auc:0.867372
[500]	Train-auc:0.90229	Val-auc:0.873793
[600]	Train-auc:0.907897	Val-auc:0.878413
[700]	Train-auc:0.912111	Val-auc:0.881954
[800]	Train-auc:0.915415	Val-auc:0.883957
[900]	Train-auc:0.918021	Val-auc:0.886025
[1000]	Train-auc:0.92047	Val-auc:0.88752
[1100]	Train-auc:0.922277	Val-auc:0.889028
[1200]	Train-auc:0.923856	Val-auc:0.89006
[1300]	Train-auc:0.925337	Val-auc:0.890738
[1400]	Train-auc:0.926461	Val-auc:0.891506
[1500]	Train-auc:0.927479	Val-auc:0.892112
[1600]	Train-auc:0.928539	Val-auc:0.892589
Stopping. Best iteration:
[1622]	Train-auc:0.928703	Val-auc:0.892691



In [ ]:
#model = RandomForestClassifier()
#model.fit(train_X, train_y)

In [ ]:
model = lgb.LGBMClassifier(max_depth=-1,n_estimators=999999,learning_rate=0.02,metric='auc',n_jobs=1)
model.fit(train_X, train_y,early_stopping_rounds=5, eval_set=[(val_X, val_y)],verbose=100)

In [ ]:
model = XGBClassifier(objective='binary:logistic', max_depth=2,n_estimators=4000, learning_rate=0.0936165921314771,n_jobs=2,min_child_weight= 53,colsample_bytree=0.3561271102144279,gamma=9.943467991283027)
model.fit(train_X, train_y,early_stopping_rounds=50, eval_metric='auc',eval_set=[(train_X,train_y),(val_X, val_y)],verbose=100)

In [22]:
#Validation
val_pred = Model.predict(xgboost.DMatrix(val_X))
val_pred2 = Model.predict(xgboost.DMatrix(train_X))
#print("Test score: {:<8.5f}".format(roc_auc_score(val_pred,val_y)))
#print("Train score: {:<8.5f}".format(roc_auc_score(val_pred2,train_y)))

In [ ]:
#Validation
val_pred = Model.predict(val_X)
val_pred2 = Model.predict(train_X)
print("Test score: {:<8.5f}".format(roc_auc_score(val_pred,val_y)))
print("Train score: {:<8.5f}".format(roc_auc_score(val_pred2,train_y)))

Submission Writer

In [24]:
print("Test_df:")
print(test_df.head())

Test_df:
  ID_code    var_0    var_1    var_2   var_3    var_4   var_5   var_6  \
0  test_0  11.0656   7.7798  12.9536  9.4292  11.4327 -2.3805  5.8493   
1  test_1   8.5304   1.2543  11.3047  5.1858   9.1974 -4.0117  6.0196   
2  test_2   5.4827 -10.3581  10.1407  7.0479  10.2628  9.8052  4.8950   
3  test_3   8.5374  -1.3222  12.0220  6.5749   8.8458  3.1744  4.9397   
4  test_4  11.7058  -0.1327  14.1295  7.7506   9.1035 -8.5848  6.8595   

     var_7   var_8   ...     var_190  var_191  var_192  var_193  var_194  \
0  18.2675  2.1337   ...     -2.1556  11.8495  -1.4300   2.4508  13.7112   
1  18.6316 -4.4131   ...     10.6165   8.8349   0.9403  10.1282  15.5765   
2  20.2537  1.5233   ...     -0.7484  10.9935   1.9803   2.1800  12.9813   
3  20.5660  3.3755   ...      9.5702   9.0766   1.6580   3.5813  15.1874   
4  10.6048  2.9890   ...      4.2259   9.1723   1.2835   3.3778  19.5542   

   var_195  var_196  var_197  var_198  var_199  
0   2.4669   4.3654  10.7200  15.4722  -8.7197

In [25]:
X_Test = test_df[features]
X_Test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [27]:
Submission_pred = Model.predict(xgboost.DMatrix(X_Test))
#val_pred3 = model.predict(X)
#print("Prediction_TEST_Target ratio:",sum(Submission_pred)/len(Submission_pred))
#print("Prediction_Train_Target ratio:",sum(val_pred3)/len(val_pred3))
#print("Train_data_Target ratio:",sum(y)/len(y))

In [28]:
import datetime
app=xw.App(visible=True,add_book=False)
wb=app.books.open(r'D:\Kaggler\Santander\submission.csv')
wb.sheets['submission'].range('B2').value=np.vstack(np.array(Submission_pred))
now=datetime.datetime.now()
File_name="submission"+"_"+now.strftime("%H%M")+"_"+str(datetime.date.today())+'.csv'
Route_name='D:\\Kaggler\\Santander\\'+File_name
wb.save(Route_name)
wb.close()
app.quit()

In [ ]:
model = RandomForestClassifier()
model.fit(train_X, train_y)


In [ ]:
preds_val = model.predict(val_X)
roc_auc_score(preds_val,val_y)

Fitting TEST

In [ ]:
def get_auc(nodes, train_X, val_X, train_y, val_y):
    model = XGBClassifier(n_estimators=10, learning_rate=0.1,max_leaf_nodes=nodes)
    model.fit(train_X, train_y,early_stopping_rounds=100,eval_metric='auc',eval_set=[(val_X, val_y)],verbose=True)
    preds_val = model.predict(val_X)
    auc = roc_auc_score(preds_val, val_y)
    return(auc)
# compare auc with differing values of max_leaf_nodes
for nodes in [3, 10, 15, 50]:
    my_auc = get_auc(nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(nodes, my_auc))

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(14, 6))
sns.distplot(train_df['target'])

In [ ]:
#Create group for target less than -30
train_df['Bad_guy']=train_df['target'].apply(lambda x: 1 if x<-30 else 2)

In [ ]:
#Create time column
train_df['period']=(max(train_df['first_active_month'])-train_df['first_active_month']).dt.days

In [ ]:
train_df.head()

In [ ]:
temp=train_df.copy()
temp=temp.drop(columns=['feature_1', 'feature_2'])
temp.corr()

In [ ]:
# checking missing data
total = train_df.isnull().sum().sort_values(ascending = False)
percent = (train_df.isnull().sum()/train_df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# checking missing data
total = test_df.isnull().sum().sort_values(ascending = False)
percent = (test_df.isnull().sum()/test_df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
my_plots = plot_partial_dependence(model,       
                                   features=[0,1,2,3,4], # column numbers of plots we want to show
                                   X=train_X,            # raw predictors data.
                                   feature_names=['A1','A2','A3','A4','A5'], # labels on graphs
                                   grid_resolution=10) # number of values to plot on x axis